In [ ]:
import geopandas as gpd
import numpy as np
import geoviews as gv
import hvplot.pandas

from shapely.geometry import LineString, Point
from scipy.interpolate import splev, splrep, BSpline

gv.extension('bokeh')

# module 

## 境界線上の格子点を設定
線上の格子点の配置を決める。
河道中心線上に等間隔に配置した点から境界線上への垂線との交点を一次配置として設定する。

In [ ]:
def mkboundaryPoints2(points, tgline, side='Left', length=500):
    outPoint = []
    outLine = []
    
    porg = points[0]
    cp = Point(tgline.coords[:][0])
    outLine.append(LineString([porg, cp]))
    outPoint.append(cp)
    
    for i in range(1, len(points)-1):
        porg = points[i]
        p1 = points[i+1]
        p0 = points[i-1]
        
        vec = [(p1.x - p0.x), (p1.y - p0.y)]
        evec = vec / np.linalg.norm(vec)
        th = 90 if side=='Left' else -90 if side=='Right' else 'error'
    
        theta =  float(th)/float(180)*np.pi
        rotvx = np.cos(theta)*evec[0] - np.sin(theta)*evec[1]
        rotvy = np.sin(theta)*evec[0] + np.cos(theta)*evec[1]
        evec = np.array([rotvx, rotvy])
        
        vec1 = [(cp.x - porg.x), (cp.y - porg.y)]
        evec1 = vec1 / np.linalg.norm(vec1)
        dot_product = np.dot(evec, evec1)
        angle = np.arccos(dot_product)
        evecZ = np.append(evec,0)
        evec1Z = np.append(evec1,0)
        nor = np.cross(evecZ, evec1Z)
        deg = np.sign(nor[2]) * angle/np.pi*float(180)
        print(i, deg)
#         if deg < 0:
#             thstart = float(0)
#         else:
#             thstart = deg
            
        thstart = deg
        thend = -90 if side=='Left' else 90 if side=='Right' else 'error'
        dth = -0.1 if side=='Left' else 0.1 if side=='Right' else 'error'
            
        # 0.1度づつ回転させて最も直交に近い点を抽出
        cpoint = []
        cangle = []
        for th in np.arange(thstart, thend, dth):
            theta =  float(th)/float(180)*np.pi
            rotvx = np.cos(theta)*evec[0] - np.sin(theta)*evec[1]
            rotvy = np.sin(theta)*evec[0] + np.cos(theta)*evec[1]
            rotevec = np.array([rotvx, rotvy])
            xl = np.array([np.array(porg.coords[:][0]), porg + length*rotevec])
            Ls = LineString(xl)
            
            if tgline.intersects(Ls):
                pcross = tgline.intersection(Ls)
                if pcross.geom_type == 'MultiPoint' :
                    cp = pcross[0]
                    cpoint.append(cp)
                    lproj = tgline.project(cp)
                else:
                    cp = pcross
                    cpoint.append(cp)
                    lproj = tgline.project(cp)
                
                dL = 0.1
                ps1 = tgline.interpolate(lproj + dL)
                ps0 = tgline.interpolate(lproj - dL)
                vecs = [(ps1.x - ps0.x), (ps1.y - ps0.y)]
                evecs = vecs / np.linalg.norm(vecs)
                
                dot_product = np.dot(rotevec, evecs)
                angle = np.arccos(dot_product)
                deg = angle/np.pi*float(180)
                cangle.append(deg)
                
                if np.abs(deg-90)<0.1 : break
        else:
            ind = np.abs( np.array(cangle) - 90).argmin()
            cp = cpoint[ind]
            
        outLine.append(LineString([porg, cp]))
        outPoint.append(cp)
    
    porg = points[-1]
    cp = Point(tgline.coords[:][-1])
    outLine.append(LineString([porg, cp]))
    outPoint.append(cp)
    
    return outPoint, outLine

## 再配置

隣り合う2点との距離が同程度になるように格子点を再配置する。

In [ ]:
def relocate(tgpoints, tgline, nn=5):
    P = tgpoints.copy()
    for _ in range(nn):
        Pout = P.copy()
    
        for i in range(1, len(P)-1):
            L0 = tgline.project(Pout[i-1])
            L1 = tgline.project(Pout[i]) 
            L2 = tgline.project(Pout[i+1])
            
            Pout[i] = tgline.interpolate(0.5*L2 + 0.5*L0)
        
        P = Pout.copy()
        
    return P

# main 

## common

In [ ]:
gdf = gpd.read_file('linesmooth.geojson', encoding='UTF-8')

In [ ]:
clines = gdf[gdf['name'] == 'Cline']['geometry'].squeeze()
llines = gdf[gdf['name'] == 'Lline']['geometry'].squeeze()
rlines = gdf[gdf['name'] == 'Rline']['geometry'].squeeze()

In [ ]:
L = np.linspace(0, clines.length, 100, endpoint=True)
points = [clines.interpolate(ll) for ll in L]

##  set mesh-point on boundary line

In [ ]:
# left-side
LP, L = mkboundaryPoints2(points, llines, side='Left', length=200)
LeftP = relocate(LP, llines, 5)

# right-side
RP, L = mkboundaryPoints2(points, rlines, side='Right', length=200)
RightP = relocate(RP, rlines, 5)

In [ ]:
LL  = [LineString([p0, p1]) for p0, p1 in zip(LeftP, RightP)]
gdfnew = gpd.GeoDataFrame({'geometry':LL, 'I': list(range(len(LL)))})
gdfnew.crs = gdf.crs
out = gdfnew.to_file("JLines.geojson", driver='GeoJSON')
del out

#  check

In [ ]:
g0 = gdfnew.hvplot()
back = gv.WMTS('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name="GoogleMapsImagery")
gall = g0*back
gallo = gall.options(width=600,height=500)

In [ ]:
gallo